In [63]:
import pandas as pd
import re
import os

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from nltk.corpus import words

# nltk.download('stopwords')

## Read Data from File

In [3]:
file_path = os.path.join(os.getcwd(), '..', 'Data','1_Clean.csv')
df = pd.read_csv(file_path)

In [4]:
df.head()

,review,sentiment
0,"Unique designs This book has beautiful photos,...",positive
1,Great Book Loved their approach in this book a...,positive
2,Five Stars great,positive
3,"Great Book! Always love the way Eva thinks, an...",positive
4,Five Stars Nice patterns,positive


In [7]:
# df['review'][100:200]

In [31]:
from bs4 import BeautifulSoup    #html tags
import unidecode                 #accented chars
import contractions              #expand contractions

# !pip install bs4
# !pip install contractions

In [ ]:
# nlp = spacy.load('en_core_web_md')

In [9]:
def lowercasing(text):
    ## changing all words to lowercase
    return text.lower()

In [18]:
def remove_html_tags(text):
    ## Remove any html tags from reviews using BeautifulSoup and lxml parser
    soup = BeautifulSoup(text, 'lxml')
    return soup.get_text(separator=" ")

In [18]:
def expand_contractions(text):      #https://medium.com/@lukei_3514/dealing-with-contractions-in-nlp-d6174300876b
    ## expand contractions and abbrevations like don't to do not
    return contractions.fix(text)

In [71]:
word_2_number_dict = {'zero': '0', 'one': '1', 'two': '2',   'three': '3', 'four': '4',
                      'five': '5', 'six': '6', 'seven': '7', 'eight': '8', 'nine': '9'}

def word_2_number(text):
    ## Convert words into numbers like one to 1
    text = text.split()
    return ' '.join(word_2_number_dict.get(word, word) for word in text)

In [51]:
def remove_digits(text):
    return ''.join(word for word in text if not word.isdigit())

In [ ]:
def remove_accented_characters(text):
    ## remove accented characters from review like Boutiqué or using unicode module
    return unidecode.unidecode(text)

In [21]:
def remove_special_characters(text):
    ## remove non-alphanumeric (\w - [a-zA-Z0-9_], \s - spaces) 
    return re.sub(r'[^\w\s]', '', text)

In [54]:
def remove_whitespaces(text):
    ## remove extra whitespaces 
    return ' '.join(text.strip().split())

In [57]:
def create_tokens(text):
    ## Create Tokens using nltk. Alternatives: Spacy, torchtext
    return nltk.word_tokenize(text)

In [ ]:
stop_words = set(stopwords.words("english")) - {'no', 'nor', 'not'}

def remove_stopwords(tokens):
    ## removing stopwords using nltk
    return [token for token in tokens if token not in stop_words]


In [83]:
# ['no', 'not']
# stop_words = stopwords.words("english")
# negative_stopwords = ['no', 'nor', 'not']
# for negative_stopword in negative_stopwords:
#     stop_words.remove(negative_stopword)    



176
<class 'set'>


In [ ]:
Lemmatization/ stemming

In [56]:
sample = df.iloc[2]['review']
sample = 'Five stars not four and its good better than 1 enough '
print('*', sample, '*', sep='')

sample = lowercasing(sample)
sample = word_2_number(sample)
sample = remove_digits(sample)
sample = remove_whitespaces(sample)

print('*', sample, '*', sep='')

*Five stars not four and its good better than 1 enough *
*stars not and its good better than enough*


In [17]:
sample = ' This is   beautiful 1 two 3  '
print('*', sample, '*', sep='')
print('*',' '.join(sample.strip().split()),'*', sep='')
print('*', ''.join(word for word in sample if not word.isdigit()), '*', sep='')

* This is   beautiful 1 two 3  *
*This is beautiful 1 two 3*
* This is   beautiful  two   *


In [20]:
remove_digits(sample)

' This is   beautiful  two   '

In [31]:
import torch

# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    # Print the number of available GPUs
    print(f"Number of available GPUs: {torch.cuda.device_count()}")
    
    # Get the name of the GPU
    gpu_name = torch.cuda.get_device_name(0)
    print(f"GPU Name: {gpu_name}")
    
    # Create a tensor on the GPU
    device = torch.device("cuda")
    tensor_on_gpu = torch.randn((3, 3)).to(device)
    
    # Print the tensor on the GPU
    print("Tensor on GPU:")
    print(tensor_on_gpu)
else:
    print("CUDA is not available. Please check your GPU and CUDA installation.")


Number of available GPUs: 1
GPU Name: NVIDIA GeForce RTX 3060 Laptop GPU
Tensor on GPU:
tensor([[ 0.7417, -0.0793, -0.3640],
        [-0.2944, -2.5446,  1.5457],
        [ 0.8345, -0.4816, -0.3052]], device='cuda:0')


In [24]:
acc = df['review'].apply(lambda x: re.findall(r'\b\w*[^\x00-\x7F]+\w*\b', x))


In [22]:
import tensorflow as tf

# Check if GPU is available
if tf.test.is_gpu_available():
    # Print information about available GPUs
    physical_devices = tf.config.list_physical_devices('GPU')
    print(f"Number of available GPUs: {len(physical_devices)}")
    
    # Print the name of the GPU
    gpu_name = tf.test.gpu_device_name()
    print(f"GPU Name: {gpu_name}")
    
    # Create a tensor on the GPU
    tensor_on_gpu = tf.random.normal((3, 3))
    
    # Print the tensor on the GPU
    print("Tensor on GPU:")
    print(tensor_on_gpu.numpy())
else:
    print("GPU is not available. Please check your TensorFlow and GPU installation.")


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is not available. Please check your TensorFlow and GPU installation.


In [4]:
df['sentiment'].value_counts()

positive    7400867
negative    1870734
Name: sentiment, dtype: int64

More positive reviews compared to the negative ones

In [12]:
file_path = os.path.join(os.getcwd(), '..', 'Data','2_PreProcessed.csv')
df.to_csv(file_path, index=False)